In [3]:
# pip install keras-facenet

In [1]:
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model
import os
from mtcnn import MTCNN
import tensorflow as tf
from scipy.spatial.distance import cosine
from datetime import datetime
from keras_facenet import FaceNet
import gc

In [ ]:
embedder = FaceNet()
detector = MTCNN()

In [11]:
path = 'dataset1/img/'
list_dir = os.listdir(path)


ids = []
persons = {}
persons_embedding = {}

# Load the images of all persons to be recognized

# person1 = cv2.imread('person1.jpg')
# person1_embedding = None

for img in list_dir:
    ids.append(img.replace(".jpg", ""))
    persons[img.replace(".jpg", "")] = cv2.imread(path+img)
    persons_embedding[img.replace(".jpg", "")] = None

for k in persons.keys():
    person = persons[k]
    person = cv2.resize(person, (160, 160))
    person = np.expand_dims(person, axis=0)
    person = (person - 127.5) / 128.0
    persons_embedding[k] = embedder.embeddings(person)

# print(persons)
# print(persons_embedding)

1/1 [==============================] - 0s 72ms/step


In [12]:
cap = cv2.VideoCapture('dataset1/Video_clip/VID_20230302_170157.mp4')
names = []
detected = []
cnt = 0
speed = 10 # frames to be skiped

w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(w),int(h))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4', fourcc, int(fps/speed), (400, 400))

while cap.isOpened():
    # Read a frame from the video stream
    ret, frame = cap.read()
    frame = cv2.resize(frame, (400, 400))
    
    if int(cnt%speed) == 0:
        cnt+=1
    else:
        cnt+=1
        continue
        
    if ret:
        # Detect faces in the frame using MTNN
        result = detector.detect_faces(frame)
        # For each face detected, extract the face embedding and compare it to the embeddings of all persons
        for face in result:
            x, y, w, h = face['box']
            x1, y1 = max(0, x), max(0, y)
            x2, y2 = min(frame.shape[1], x+w), min(frame.shape[0], y+h)
            face_img = frame[y1:y2, x1:x2]
            face_img = cv2.resize(face_img, (160, 160))
            face_img = np.expand_dims(face_img, axis=0)
            face_img = (face_img - 127.5) / 128.0  # make range of -1 to 1
            face_embedding = embedder.embeddings(face_img)
            # print(face_embedding)

            # Compare the face embedding to the embeddings of all persons
            for ID in ids:
                if ID not in detected:
                    if cosine(face_embedding[0], persons_embedding[ID][0]) < 0.4:
                        detected.append(ID)
                        names.append([datetime.now(),ID])
            cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), thickness=2)
            del(x)
            del(y)
            del(w)
            del(h)
            del(x1)
            del(y1)
            del(x2)
            del(y2)
            del(face_img)
            del(face_embedding)
            gc.collect()
            # del()
        cv2.imshow("clip", frame)
        out.write(frame)
        
        if (cv2.waitKey(1) & 0xFF) == ord('f'):
            break
cap.release()
out.release()
cv2.destroyAllWindows()
# print(names)
df = pd.DataFrame(names, columns=["DateTime", "ID"])
df.to_csv("attendence.csv")
print(df)

1/1 [==============================] - 0s 93ms/step
                     DateTime  ID
0  2023-03-05 10:32:33.182487   1
1  2023-03-05 10:32:33.182487  10
2  2023-03-05 10:32:33.182487  11
3  2023-03-05 10:32:33.182487  12
4  2023-03-05 10:32:33.183461  13
5  2023-03-05 10:32:33.183461  14
6  2023-03-05 10:32:33.183461  15
7  2023-03-05 10:32:33.183461  16
8  2023-03-05 10:32:33.183461  17
9  2023-03-05 10:32:33.183461  18
10 2023-03-05 10:32:33.183461   2
11 2023-03-05 10:32:33.183461  20
12 2023-03-05 10:32:33.183461  22
13 2023-03-05 10:32:33.183461  23
14 2023-03-05 10:32:33.183461  24
15 2023-03-05 10:32:33.183461  25
16 2023-03-05 10:32:33.183461  26
17 2023-03-05 10:32:33.183461  27
18 2023-03-05 10:32:33.183461  28
19 2023-03-05 10:32:33.183461  29
20 2023-03-05 10:32:33.183461   3
21 2023-03-05 10:32:33.183461  30
22 2023-03-05 10:32:33.184473   4
23 2023-03-05 10:32:33.184547   5
24 2023-03-05 10:32:33.184547   6
25 2023-03-05 10:32:33.184547   7
26 2023-03-05 10:32:33.184547 

In [4]:
cap = cv2.VideoCapture('dataset1/Video_clip/VID_20230302_170157.mp4')
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS) 
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (int(w),int(h)))
while cap.isOpened():
    # Read a frame from the video stream
    ret, frame = cap.read()
    if ret:
        cv2.imshow("clip", frame)
        out.write(frame)
        if (cv2.waitKey(1) & 0xFF) == ord('f'):
            break
cap.release()
out.release()
cv2.destroyAllWindows()

https://github.com/arsfutura/face-recognition

https://www.kaggle.com/code/yhuan95/face-recognition-with-facenet/input?select=facenet_keras.h5

https://www.sitepoint.com/keras-face-detection-recognition/#:~:text=In%20the%20first%20step%20of,belong%20to%20the%20same%20person.